In [1]:
import tensorflow as tf
import os
import numpy as np
import math, numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras
from tensorflow.keras.optimizers import RMSprop
from pathlib import Path

from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML, display, clear_output, SVG

In [2]:
Path('/content/weights').mkdir(parents=True, exist_ok=True)

# Setting up dataset

In [3]:
!git clone https://github.com/Sern5011/Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition.git

Cloning into 'Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition'...
remote: Enumerating objects: 83777, done.
remote: Counting objects: 100% (26406/26406), done.
remote: Compressing objects: 100% (26399/26399), done.

remote: Total 83777 (delta 7), reused 26395 (delta 7), pack-reused 57371
Receiving objects: 100% (83777/83777), 12.63 GiB | 30.67 MiB/s, done.
Resolving deltas: 100% (31/31), done.
Checking out files: 100% (83743/83743), done.


In [11]:
base_dir = './Automatic-item-checkout-system-for-supermarkets-based-on-object-detection-and-recognition/dataset/'
train_dir = os.path.join(base_dir, 'train2019')
test_dir = os.path.join(base_dir, 'test2019')

In [15]:
# Updated to do image augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        base_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 83739 images belonging to 3 classes.


# VGG16

In [24]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3))



# Load the CONV layers of VGG16 model and add the FC layers

model = models.Sequential()
model.add(conv_base)
#model.pop()
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [25]:
conv_base.summary()

for layer in conv_base.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in conv_base.layers:
    print(layer, layer.trainable)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])